In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install pascal-voc-writer

MessageError: ignored

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15155, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15155 (delta 7), reused 9 (delta 3), pack-reused 15138
Receiving objects: 100% (15155/15155), 14.15 MiB | 16.69 MiB/s, done.
Resolving deltas: 100% (10385/10385), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.3.2 which is incompatible.


In [ ]:
!cp /content/drive/MyDrive/deeplearnin_dataset/Objects_.zip /content/
!unzip /content/Objects_.zip

In [ ]:
import os

old_file_name = "/content/yolov5/Objects_/data.yaml.txt"
new_file_name = "/content/yolov5/Objects_/data.yaml"

os.rename(old_file_name, new_file_name)

In [ ]:
!cp /content/drive/MyDrive/Best/best.pt /content/

In [ ]:
%cd /content/yolov5
!python train.py --img 1280 --batch 4 --epochs 100 --data '/content/yolov5/Objects_/data.yaml' --weights "yolov5l6.pt" --name yolov5s_results --project '/content/football' --cache
#!python train.py --img 640 --batch 4 --epochs 2 --data '/content/yolov5/Objects_/data.yaml' --weights "/content/best.pt" --cfg yolov5l.yaml --name yolov5s_results --project '/content/football' --cache
#!python train.py --img 640 --batch 4 --epochs 1 --data '/content/yolov5/Objects_/data.yaml' --weights "/content/best.pt" --name yolov5s_results --project '/content/football' --cache --resume

In [ ]:
import shutil
shutil.copy("/content/football/yolov5s_results/weights/best.pt","/content/drive/MyDrive/Test")

In [ ]:
files.download('/content/football/yolov5s_results/weights/best.pt')


In [ ]:
import os
os.makedirs('/content/yolov5/Football_images/XML', exist_ok=True)

In [ ]:
import torch
from PIL import Image
import pandas as pd
import cv2
from pascal_voc_writer import Writer
from google.colab import files
import shutil

# Model

model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/best.pt', force_reload=True)
model.conf = 0.50
for i in filename:
  
  results = model(Image.open(i))

  results.render()
  torch.cuda.empty_cache()

  df=results.pandas().xyxy[0]
  df=df.drop(['confidence'],axis=1)
  df=df[['class','xmin','xmax','ymin','ymax','name']]
  count_row = df.shape[0]
  

  im = cv2.imread(i)
  height=im.shape[0]
  width=im.shape[1]
  depth=im.shape[2]

  Name_temp = i.stem + '.jpg'
  Name_xml = i.stem + '.xml'

  # create pascal voc writer (image_path, width, height)
  writer = Writer(Name_temp, width, height,depth)

  # add objects (class, xmin, ymin, xmax, ymax)
  for i in range(0,count_row):
      writer.addObject(df["name"].values[i], df["xmin"].values[i], df["ymin"].values[i], df["xmax"].values[i], df["ymax"].values[i])
  #writer.addObject('person', 40, 90, 100, 150)

  # write to file
  writer.save('/content/yolov5/Football_images/XML/' + Name_xml)


In [ ]:
shutil.make_archive('/content/yolov5/Football_images/XML', 'zip', '/content/yolov5/Football_images/XML')
files.download('/content/yolov5/Football_images/XML.zip')
